In [2]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import xarray as xr
import matplotlib.pyplot as plt
import sys
sys.path.append(r'C:\Users\dpoppema\OneDrive - Delft University of Technology\Documents\GitHub\HybridDune\Ruben\Pressure_sensors\S1\RBR_05')
from scipy.signal import welch
from scipy.ndimage import uniform_filter1d
import puv 
from copy import copy

In [3]:
def solo_data_reader(dataFile, sf):
    '''
    Function to read solo datafile.
    Returns a dataframe with a time column and pressure column in Pascal
    '''
    p = []
    datt = []
    with open(dataFile) as myfile:
        for index, line in enumerate(myfile):
            if index >= 1:
                lin = line.split(',')
                datt.append(lin[0])
                p.append(float(lin[1]))
    p = np.array(p) * 1e4  # dBar to Pa

    t = pd.date_range(datt[0], periods=len(datt), freq='{}s'.format(1 / sf))

    dfp = pd.DataFrame(data={'p': p}, index=t)

    dfp.index.name = 't'
    return dfp

In [4]:
def movmean(x, N):
    # calculate moving mean. NB: this divides values at the edges by the window length, instead of the available number of values
    y = uniform_filter1d(x, size=N, mode='constant') # for even window: backward avg. So window 2: x_m(i)=[x(i-1)+x(i)]/2. x_m(i=1) = x(i=1)/2

    # compensate edges for number of values, i.e. the truncated window length
    S1 = np.arange(np.ceil(N/2), N)
    S2 = np.ones(len(x)-N+1)*N
    S3 = np.arange(N-1, np.floor(N/2), -1)
    S = np.concatenate((S1, S2, S3)) 
    return y * N / S
    # for 2D inspiration, see https://stackoverflow.com/questions/23000260/numpy-two-dimensional-moving-average

In [5]:
# # LOAD RAW DATA, SAVE TO NETCDF
# Physical constants -----------------------------------------------------------------------------------
rho = 1027 #kg/m3, for seawater at 9C, avg temp at HKZ measurement station
g = 9.8125  # value Zandmotor

# # input parameters per file ----------------------------------
subfolder_in_all = ['refP1 RBR3', 'S3P3 RBR6', 'S1P2 RBR2', 'S3P2 RBR4', 'S4P2 RBR1','S3P1 RBR5'] # subfolder where file is sitting within experimentFolder (on O drive Daan)
instrumentname_all = subfolder_in_all 
namedatafile_all = ['RBR3 ref 2024-12-31 to 2025-01-24_11,30_data.txt',       # name of the datafile
                    'RBR6 S2F1 2024-12-31 to 2025-01-24_11,12_data.txt',
                    'RBR2 S1F2 2025-01-06 to 2025-01-16_15,00_data.txt',
                    'RBR4 S3F2 2024-12-31 to 2025-01-16_15,41_data.txt',
                    'RBR1 S4F2 2025-01-06 to 2025-01-16_14,58_data.txt',
                    'RBR5 S2F4 2024-12-31 to 2025-01-16_15,41_data.txt']                                                          
#                 Ref.P1 RBR3,  S3P3 RBR6,  S1P2 RBR2,  S3P2 RBR4,  S4P2 RBR1,  S3P1 RBR5
sf_all            = [       8,         16,          8,          8,          8,         16]              # [hz] sampling frequency
offset_all        = [      64,         38,        320,       -543,        268,       -147]              # [Pa] offset to be added to the pressure data, for instrument calibration
xRD_all           = [  np.nan,  72398.360,  72444.936,  72412.935,  72398.537,  72446.111]              # x position of placement in field
yRD_all           = [  np.nan, 452130.334, 452163.098, 452119.047, 452098.880, 452094.373]              # y position of placement in field
zi_all            = [  np.nan,     -0.747,      0.427,     np.nan,     -0.073,      1.171]              # z position of instrument (sensor) in field
serial_number_all = ['202440',   '208682',   '202439',   '202441',   '202438',   '208681']              # unique serial number of the instrument
t_installed_all   = pd.to_datetime(['2024-12-31 11:00',  '2024-12-31 11:00', '2025-01-06 14:00', '2024-12-31 14:00', '2025-01-06 14:00', '2024-12-31 14:00'])
t_removed_all     = pd.to_datetime(['2025-01-18 09:00', '2025-01-18 09:00', '2025-01-14 08:00', '2025-01-14 08:00', '2025-01-14 08:00', '2025-01-14 08:00'])
zi_i3 = [0.379, 0.090] # S3F2 RBR4 (i=3) was moved down
t_zi_i3 = pd.to_datetime(['2024-12-31 10:30',  '2025-01-06 12:00'])

zb_i1   = np.array( [      -1.164,                                  -1.417                                                                             ] ) # S3P3 RBR6
zb_i2   = np.array( [                                                                     -0.082,              -0.284,              -0.603             ] ) # S1P2 RBR2
zb_i3   = np.array( [      -0.093,               0.088,             -0.371,               -0.399,              -0.363,              -0.7               ] ) # S3P2 RBR4
zb_i4   = np.array( [                                                                     -0.576,              -0.344,              -0.693             ] ) # S4P2 RBR1
zb_i5   = np.array( [       0.92,                0.646,              0.642,                0.759,               0.679                                  ] ) # S3P1 RBR5

t_zb_i1 = pd.to_datetime([ '2024-12-31 10:30',                      '2025-01-04 12:50'                                                                 ]) # S3F1 RBR6
t_zb_i2 = pd.to_datetime([                                                                '2025-01-06 12:00',  '2025-01-08 13:20',  '2025-01-12 18:45' ]) 
t_zb_i3 = pd.to_datetime([ '2024-12-31 10:30',  '2025-01-03 12:00',  '2025-01-04 12:50',  '2025-01-06 12:00',  '2025-01-08 13:20',  '2025-01-12 18:45' ]) 
t_zb_i4 = pd.to_datetime([                                                                '2025-01-06 12:00',  '2025-01-08 13:20',  '2025-01-12 18:45' ]) 
t_zb_i5 = pd.to_datetime([ '2024-12-31 10:30',  '2025-01-03 12:00',  '2025-01-04 12:50',  '2025-01-06 12:00',  '2025-01-08 13:20'                      ]) 

# convert RD coordinates to local coordinates
xy_RD = np.array([xRD_all, yRD_all]).T
a = np.deg2rad(36)
transformation_matrix = np.array([ [np.cos(a), np.sin(a)],[-np.sin(a), np.cos(a)] ])
xy_loc = ( xy_RD - [71683.584, 452356.055] ) @ transformation_matrix
x_loc_all = xy_loc.T[0]
y_loc_all = xy_loc.T[1]

# # input parameters general ---------------------------------------------
experimentFolder = r'O:\HybridDune experiment\data RBR, OSSI\copy RBR Udrive series2'                   # path where the data is sitting

# # Define input parameters for file i   (start loop) -----------------------
for i in [0]: #range(0,6): #RBR5 S2F4 2024-12-31 to 2025-01-16_15,41_data
    sf = sf_all[i]  #[hz] sampling frequency
    data_inDir = os.path.join(experimentFolder, subfolder_in_all[i])
    namedatafile = namedatafile_all[i]
    instrumentName = instrumentname_all[i]                                                              # designated name of the instrument
    xRD = xRD_all[i]                                                                                    # x position of placement in field
    yRD =  yRD_all[i]                                                                                   # y position of placement in field
    x_loc = x_loc_all[i]                                                                                # x position of instrument in local coordinate system [m]
    y_loc = y_loc_all[i]                                                                                # y position of instrument in local coordinate system [m]
    t_installed = t_installed_all[i]
    t_removed = t_removed_all[i]

    # Define instrument height
    if i == 3: # file 3 (RBR4) was moved down at some point, has two instrument levels
        t_zi = t_zi_i3
        zi = zi_i3
    elif i>= 1: # instrument 0 is reference sensor, not installed at beach, instrument height not applicable
        zi = zi_all[i]
    
    # Define bed height
    if i ==1:
        t_zb = t_zb_i1; zb = zb_i1
    elif i==2:
        t_zb = t_zb_i2; zb = zb_i2
    elif i==3:
        t_zb = t_zb_i3; zb = zb_i3
    elif i==4:
        t_zb = t_zb_i4; zb = zb_i4
    elif i==5:
        t_zb = t_zb_i5; zb = zb_i5

    # UNCOMMENT SECTION TO LOAD RAW DATA
    # # Do the reading from file and cast in xarray dataset ----------------
    # dataFile =  os.path.join(data_inDir, namedatafile)                                                 # path + name datafile
    # dfp = solo_data_reader(dataFile, sf)
    # ds = dfp.to_xarray()
    
    # # add instrument height
    # if i == 3:                      # file 3 (RBR4) was moved down at some point, has two instrument levels.
    #     ds['t_zi'] = t_zi           # assign dimension & var t_zi (t_zi is a vector, so this syntax adds a dimension instead of variable)
    #     ds['zi'] = ('t_zi', zi)     # assign variable zi, with coordinate t_zi and value zi
    #     ds.t_zi.attrs = {'long name': 'time that instrument elevation was measured'}
    # elif i>=1:                      # instrument 0 is reference sensor, not installed at beach, instrument height not applicable
    #     ds['zi'] = zi               # assign variable zi (scalar): instrument height [m NAP]

    # # add bed height
    # if i >= 1:                  # instrument 0 is reference sensor, not installed at beach, instrument height not applicable
    #     ds['t_zb'] = t_zb           # assign dimension and var t_zb 
    #     ds['zb'] = ('t_zb',zb)      # assign variable zb (scalar): bed level [m NAP]

    # # Add instrument variables for metadata: location, frequency
    # ds['x_RD'] = xRD                # x position of instrument, in RDNAP coordinates [m]
    # ds['y_RD'] = yRD                # y position of instrument, in RDNAP coordinates [m]
    # ds['x_local'] = x_loc           # x position of instrument, in local coordinate system [m]
    # ds['y_local'] = y_loc           # y position of instrument, in local coordinate system [m] 
    # ds['sf'] = sf                   # sampling frequency [hz]
    # ds['t_installed'] = t_installed # time that the instrument was installed at the indicated height and location at the beach
    # ds['t_removed'] = t_removed     # time that the instrument was removed

    # # Add global attribute metadata
    # ds.attrs = {
    #     'Conventions': 'CF-1.6',
    #     'name': 'Pressure sensor ' + instrumentName[0:-4] + 'RBR, period 2',
    #     'instrument': 'Pressure sensor ' + instrumentName,
    #     'instrument type': 'Ruskin RBR Solo',
    #     'instrument serial number': '{}'.format(serial_number_all[i]),
    #     'epsg': 28992,
    #     'time zone': 'UTC+1',
    #     'start time': pd.to_datetime(ds.t.values[0]).strftime("%d-%b-%Y %H:%M:%S"),
    #     'end time':   pd.to_datetime(ds.t.values[-1]).strftime("%d-%b-%Y %H:%M:%S"),
    #     'summary': 'HybridDune experiment: raw pressure data',
    #     'contact person': 'Daan Poppema',
    #     'emailadres': 'd.w.poppema@tudelft.nl',
    #     'construction datetime': datetime.now().strftime("%d-%b-%Y %H:%M:%S"),
    #     'version': 'v1',
    #     'comment_1': 'constructed with xarray',
    #     'url of online dataset': 'ADD LATER'}      # DAAN: ADD URL LATER
    
    # # Add attributes to variables for metadata
    # local_coord_sys = 'x=cross-shore (positive=landward); y=alongshore (positive is to north-east); (800,200) is the southern seaward corner of the containers'
    # coord_conv   = '(0,0) local is (71683.584,452356.055) RD coordinates; local x-axis is 36° clockwise from RD x-axis; i.e. [x_loc y_loc] = [x_RD y_RD] - [x0 y0] .* [cosd(36) sind(36); -sind(36) cosd(36)]'
    # ds.p.attrs = {'units': 'Pa', 'long_name': 'pressure', 'comments': 'raw data'}
    # ds.x_RD.attrs = {'units': 'm', 'long_name': 'x position of instrument in RDNAP coordinates', 'epsg': 28992}
    # ds.y_RD.attrs = {'units': 'm', 'long_name': 'y position of instrument in RDNAP coordinates', 'epsg': 28992}
    # ds.x_local.attrs = {'units': 'm', 'long_name': 'cross-shore position of instrument in local coordinate system','local_coordinate_system': local_coord_sys, 'coordinate_conversion': coord_conv}
    # ds.y_local.attrs = {'units': 'm', 'long_name': 'alongshore position of instrument in local coordinate system','local_coordinate_system': local_coord_sys, 'coordinate_conversion': coord_conv}
    # ds.sf.attrs = {'units': 'Hz', 'long_name': 'sampling frequency'}
    # ds['t_installed'].attrs = {'long name': 'date and time that the instrument was installed at the indicated height and location at the beach'}
    # ds['t_removed'].attrs = {'long name': 'date and time that the instrument was removed'}

    # if i >= 1: # instrument 0 is reference sensor, instrument and bed height only applicable for the remaining sensors
    #     ds.zi.attrs = {'units': 'm +NAP', 'long_name': 'elevation of sensor'}  # instrument height
    #     ds.zb.attrs = {'units': 'm +NAP', 'long_name': 'bed level'}  
    #     ds.t_zb.attrs = {'long name': 'time that bed level at instrument was measured'}

    # # Save to netcdf -------------------------------
    # ds.p.values = np.round(ds.p.values)    # Round pressure to 1 Pa = 0.1 mm (file size 7 times smaller)
    # encoding = {var: {"zlib": True, "complevel": 4} for var in list(ds.data_vars) + list(ds.coords)}  # Apply deflate compression to all variables and coordinates in netCDF
    # ds.encoding = encoding  # add the encoding to the dataset (not really necessary, but allows retrieval later on)

    # ncOutDir = os.path.join(experimentFolder, 'raw NetCDF')
    # if not os.path.isdir(ncOutDir):
    #     os.mkdir(ncOutDir)
    # ds.to_netcdf(os.path.join(ncOutDir, 'Pressure sensor ' + instrumentName + ' raw data - period 2.nc'), encoding=encoding)  

In [6]:
# CALCULATE AIR PRESSURE + RELATIVE PRESSURE, SAVE TO NETCDF 
for i in range(1,6): #[0,1,2,3,4,5]:

    # Set smoothing window for atmpospheric pressure
    t_smooth_air = 10 # [s]     # measured with 8 hz. But p_water and p_air are measured up to 100m apart (and p_air inside). Affected different by wind gusts, so filter out short-term variation

    # # CALCULATE ATMOSPHERIC AIR PRESSURE ----------------------------------------------------
    dataFile =r'O:\HybridDune experiment\data RBR, OSSI\copy RBR Udrive series2\raw NetCDF\Pressure sensor refP1 RBR3 raw data - period 2.nc'
    ds = xr.open_dataset(dataFile)

    # Calibrate referense sensor
    ds['p'] = ds['p'] + offset_all[0]  # add offset to pressure data, for instrument calibration

    # crop dataset to the time range of interest. 5 RBRs with about the same end time, make exactly the same
    t0 = ds.t_installed.values          # first full hour that instrument was installed at the beach
    t_end = ds.t_removed.values         # last full hour that instrument was installed at the beach
    ds_air_8hz = ds.sel(t=slice(t0, t_end))
    tAir_8hz = ds_air_8hz['t']

    # Determine moving average. 
    pAir_smooth_8hz = movmean(ds_air_8hz['p'], 8*t_smooth_air) # smooth over 8hz * n seconds

    # interpolate for 16hz sensors
    tAir_16hz   = pd.date_range(t0, t_end, freq='{}s'.format(1 / 16)) # 16hz time vector
    pAir_smooth_16hz = np.interp(tAir_16hz, tAir_8hz, pAir_smooth_8hz) # interpolate to 16hz time vector

    # Add smoothed air pressure to temporary dataset
    ds_air_8hz['pAir_smooth'] = (('t'), pAir_smooth_8hz )
    ds_air_16hz = xr.Dataset( # make dataset for 16hz air pressure
        data_vars={'pAir_smooth': (('t',), pAir_smooth_16hz)},
        coords={'t': tAir_16hz} )

    # Load pressure of instrument, subtract air pressure ----------------------------------------------------
    instrumentName = instrumentname_all[i]                                                                               # designated name of the instrument
    print(instrumentName)
    dataFile = os.path.join(experimentFolder,'raw NetCDF','Pressure sensor ' + instrumentName + ' raw data - period 2.nc')

    ds0 = xr.open_dataset(dataFile)
    ds0 = ds0.sel(t=slice(t0, t_end))  # crop dataset to the time range of interest
    ds0 = ds0.rename({'p': 'p_abs'})   # rename p to p_abs

    # Relative pressure: correct the pressure signal with pAir
    if i == 0: # refP1 RBR3
        ds0['p_air'] = ds_air_8hz['pAir_smooth'] # air pressure, already calibrated
    elif sf_all[i] == 8:    
        ds0['p_rel'] = ds0['p_abs'] + offset_all[i] - ds_air_8hz['pAir_smooth']    # relative pressure. Add calibration offset per instrument. ds_air already calibrated 
    else:
        ds0['p_rel'] = ds0['p_abs'] + offset_all[i] - ds_air_16hz['pAir_smooth'] 

    # Correct negative pressures
    if i != 0: # only for non-reference sensors
        block_mask = ds0['p_rel'] < 0
        ds0['p_rel'] = ds0['p_rel'].where(~block_mask, 0)               # set negative pressures to zero
        
    # Add metadata
    cal_text = '{}'.format(offset_all[i]) + ' Pa added to raw pressure, based on the period 23dec, 19:11 to 19:26, during the calibration test'
    ds0.p_abs.attrs = {'units': 'Pa', 'long_name': 'Absolute pressure', 'comments': 'calibrated', 'calibration': cal_text}
    
    if i == 0: # refP1 RBR3
        cal_text = '{}'.format(offset_all[0]) + ' Pa added to raw pressure, based on the period 23dec, 19:11 to 19:26, during the calibration test'
        ds0['p_air'].attrs = {'units': 'Pa', 'long_name': 'Air pressure', 'comments': '10s moving average and calibrated','calibration': cal_text}
        ds0.attrs['summary'] = 'Hybrid-Dune campaign: air pressure'
    else:
        cal_text = '{}'.format(offset_all[i]) + ' Pa added to raw pressure of instrument (and ' + '{}'.format(offset_all[0]) + ' Pa added to air pressure of sensor refP1), based on the period 23dec, 19:11 to 19:26, during the calibration test'
        ds0['p_rel'].attrs = {'units': 'Pa', 'long_name': 'Relative pressure', 'comments': 'corrected for air pressure and calibrated','calibration': cal_text}
        ds0.attrs['summary'] = 'Hybrid-Dune campaign: pressure corrected for air pressure'

    ds0.attrs['start time'] = pd.to_datetime(ds.t.values[0]).strftime("%d-%b-%Y %H:%M:%S")
    ds0.attrs['end time'] =   pd.to_datetime(ds.t.values[-1]).strftime("%d-%b-%Y %H:%M:%S")

    # Save to netcdf -------------------------------
    ds0.p_abs.values = np.round(ds0.p_abs.values)    # Round pressure to 1 Pa = 0.1 mm
    if i == 0:    # refP1 RBR3
        ds0.p_air.values = np.round(ds0.p_air.values)    
        ncFilePath = os.path.join(experimentFolder, 'QC', 'Pressure sensor ' + instrumentName + ' p_air - period 2.nc')
    else:
        ds0.p_rel.values = np.round(ds0.p_rel.values)   
        ncFilePath = os.path.join(experimentFolder, 'QC', 'Pressure sensor ' + instrumentName + ' p_rel - period 2.nc')
    encoding = {var: {"zlib": True, "complevel": 4} for var in list(ds0.data_vars) + list(ds0.coords)}  # Apply deflate compression to all variables and coordinates in netCDF
    ds0.encoding = encoding  # add the encoding to the dataset (not really necessary, but allows retrieval later on)

    if not os.path.isdir(os.path.join(experimentFolder,'QC')):
        os.mkdir(os.path.join(experimentFolder,'QC'))
    ds0.to_netcdf(ncFilePath, encoding=encoding) # save to netcdf

S3P3 RBR6
S1P2 RBR2
S3P2 RBR4
S4P2 RBR1
S3P1 RBR5


In [7]:
ncFilePath

'O:\\HybridDune experiment\\data RBR, OSSI\\copy RBR Udrive series2\\QC\\Pressure sensor S3P1 RBR5 p_rel - period 2.nc'